In [2]:
!pip install keras==2.2.4

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 312 kB 7.6 MB/s eta 0:00:01


In [3]:
!pip install tables

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 4.3 MB 8.3 MB/s eta 0:00:01
     |████████████████████████████████| 162 kB 14.8 MB/s eta 0:00:01


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import *
from keras.initializers import RandomNormal,glorot_uniform
from keras.layers import *
from keras.models import Model, load_model
from keras.callbacks import Callback
from keras import backend as K
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tqdm import tqdm
from keras.utils import multi_gpu_model
import tensorflow as tf
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [ ]:
'''
注释的代码均是预处理代码，先运行一遍，再整体跑
'''

In [2]:

# train_data = pd.read_csv('train_data.csv',sep = '|')
# test_data_A = pd.read_csv('test_data_A.csv',sep = '|')
# test_data_B = pd.read_csv('test_data_B.csv',sep = '|')
# test_data_B['pt_d']-=1
# test_data = pd.concat([test_data_A,test_data_B]).reset_index(drop=True)
# all_data = pd.concat([train_data, test_data]).reset_index(drop=True)

# all_col=['task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
#        'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags',
#        'app_first_class', 'app_second_class', 'age', 'city', 'city_rank',
#        'device_name', 'device_size', 'career', 'gender', 'net_type',
#        'residence', 'his_app_size', 'his_on_shelf_time', 'app_score',
#        'emui_dev', 'list_time', 'device_price', 'up_life_duration',
#        'up_membership_grade', 'membership_life_duration', 'consume_purchase',
#         'communication_avgonline_30d', 'indu_name']
# for i in tqdm(all_col):
#     lbl = LabelEncoder()
#     all_data[i] = lbl.fit_transform(all_data[i])
# train_data = all_data.iloc[0:len(train_data),:]
# test_data = all_data.iloc[len(train_data):,:].reset_index(drop=True)
# train_data.to_hdf('train_data.h5', key='data')
# test_data.to_hdf('test_data.h5', key='data')

In [3]:
all_col=['task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
       'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags',
       'app_first_class', 'app_second_class', 'age', 'city', 'city_rank',
       'device_name', 'device_size', 'career', 'gender', 'net_type',
       'residence', 'his_app_size', 'his_on_shelf_time', 'app_score',
       'emui_dev', 'list_time', 'device_price', 'up_life_duration',
       'up_membership_grade', 'membership_life_duration', 'consume_purchase',
        'communication_avgonline_30d', 'indu_name']#,'expo_day_counts','expo_lastday_counts','click_lastday_count'

In [4]:
train_data = pd.read_hdf('train_data.h5', key='data')
test_data = pd.read_hdf('test_data.h5', key='data')
#all_data = pd.concat([train_data, test_data]).reset_index(drop=True)

In [7]:
# temp_data = train_data[train_data['label']==1].groupby(['uid','pt_d']).agg({'adv_id':list}).reset_index()
# temp_data = temp_data.sort_values(['uid','pt_d'])
# uid = temp_data['uid'].tolist()
# pt_d = temp_data['pt_d'].tolist()
# adv_id = temp_data['adv_id'].tolist()
# res = []
# for i in tqdm(range(0,len(uid))):
#     res.append([uid[i],pt_d[i],','.join(list(map(str,adv_id[i])))])
# total_days = 8
# result = [[res[0][0],res[0][1]+1,res[0][2]]]
# for i in tqdm(range(1,len(res))):
#     if res[i][0] == result[-1][0]:
#         while res[i][1]>result[-1][1]:
#             result.append([result[-1][0],result[-1][1]+1,result[-1][2]])
#         while res[i][1]>=result[-1][1]:
#             result.append([result[-1][0], result[-1][1] + 1, result[-1][2]+','+res[i][2]])
#     else:
#         while total_days>result[-1][1]:
#             result.append([result[-1][0],result[-1][1]+1,result[-1][2]])
#         result.append([res[i][0], res[i][1]+1, res[i][2]])
# result = pd.DataFrame(result)
# result.columns = ['uid','pt_d','his_adv_id']
# result['his_adv_id'] = result['his_adv_id'].apply(lambda x:list(map(int,x.split(','))))
# result['his_advid_len'] = result['his_adv_id'].apply(lambda x:len(x))
# train_data = train_data.merge(result,on = ['uid','pt_d'],how='left')
# test_data = test_data.merge(result,on = ['uid','pt_d'],how='left')
# train_data = train_data.fillna(0)
# test_data = test_data.fillna(0)
# his_adv_id = train_data['his_adv_id'].tolist()
# for i in tqdm(range(len(his_adv_id))):
#     if his_adv_id[i]==0:
#         his_adv_id[i] = [0]*20
#     else:
#         his_adv_id[i] =[0]*(20-len(his_adv_id[i]))+his_adv_id[i][::-1][0:20]
# his_adv_id = np.array(his_adv_id)
# test_his_adv_id = test_data['his_adv_id'].tolist()
# for i in tqdm(range(len(test_his_adv_id))):
#     if test_his_adv_id[i]==0:
#         test_his_adv_id[i] = [0]*20
#     else:
#         test_his_adv_id[i] =[0]*(20-len(test_his_adv_id[i]))+test_his_adv_id[i][::-1][0:20]
# test_his_adv_id = np.array(test_his_adv_id)
# np.save('train_his_advid.npy', his_adv_id)
# np.save('test_his_advid.npy', test_his_adv_id)

In [8]:
train_his_advid = np.load('train_his_advid.npy')
test_his_advid = np.load('test_his_advid.npy')

In [9]:
# temp_data = train_data[train_data['label']==1].groupby(['uid','pt_d']).agg({'adv_prim_id':list}).reset_index()
# temp_data = temp_data.sort_values(['uid','pt_d'])
# uid = temp_data['uid'].tolist()
# pt_d = temp_data['pt_d'].tolist()
# adv_id = temp_data['adv_prim_id'].tolist()
# res = []
# for i in tqdm(range(0,len(uid))):
#     res.append([uid[i],pt_d[i],','.join(list(map(str,adv_id[i])))])
# total_days = 8
# result = [[res[0][0],res[0][1]+1,res[0][2]]]
# for i in tqdm(range(1,len(res))):
#     if res[i][0] == result[-1][0]:
#         while res[i][1]>result[-1][1]:
#             result.append([result[-1][0],result[-1][1]+1,result[-1][2]])
#         while res[i][1]>=result[-1][1]:
#             result.append([result[-1][0], result[-1][1] + 1, result[-1][2]+','+res[i][2]])
#     else:
#         while total_days>result[-1][1]:
#             result.append([result[-1][0],result[-1][1]+1,result[-1][2]])
#         result.append([res[i][0], res[i][1]+1, res[i][2]])
# result = pd.DataFrame(result)
# result.columns = ['uid','pt_d','his_adv_prim_id']
# result['his_adv_prim_id'] = result['his_adv_prim_id'].apply(lambda x:list(map(int,x.split(','))))
# train_data = train_data.merge(result,on = ['uid','pt_d'],how='left')
# test_data = test_data.merge(result,on = ['uid','pt_d'],how='left')
# train_data = train_data.fillna(0)
# test_data = test_data.fillna(0)
# his_adv_id = train_data['his_adv_prim_id'].tolist()
# for i in tqdm(range(len(his_adv_id))):
#     if his_adv_id[i]==0:
#         his_adv_id[i] = [0]*20
#     else:
#         his_adv_id[i] =[0]*(20-len(his_adv_id[i]))+his_adv_id[i][::-1][0:20]
# his_adv_id = np.array(his_adv_id)
# test_his_adv_id = test_data['his_adv_prim_id'].tolist()
# for i in tqdm(range(len(test_his_adv_id))):
#     if test_his_adv_id[i]==0:
#         test_his_adv_id[i] = [0]*20
#     else:
#         test_his_adv_id[i] =[0]*(20-len(test_his_adv_id[i]))+test_his_adv_id[i][::-1][0:20]
# test_his_adv_id = np.array(test_his_adv_id)
# np.save('train_his_adv_prim_id.npy', his_adv_id)
# np.save('test_his_adv_prim_id.npy', test_his_adv_id)

In [10]:
train_his_adv_prim_id = np.load('train_his_adv_prim_id.npy')
test_his_adv_prim_id = np.load('test_his_adv_prim_id.npy')

In [11]:
# all_data = pd.concat([train_data, test_data]).reset_index(drop=True)
# params = {'feature_config':{}}
# for i in tqdm(all_col):
#     params['feature_config'][i] = {}
#     params['feature_config'][i]['vocabulary_size'] = all_data[i].max()+1
# #     if i in {'task_id','adv_id'}:
# #         params['feature_config'][i]['embedding_dim'] = 32
# #     else:
#     params['feature_config'][i]['embedding_dim'] = 16
        

In [3]:
params={'feature_config': {'task_id': {'vocabulary_size': 4932, 'embedding_dim': 16},
  'adv_id': {'vocabulary_size': 5956, 'embedding_dim': 16},
  'creat_type_cd': {'vocabulary_size': 8, 'embedding_dim': 16},
  'adv_prim_id': {'vocabulary_size': 114, 'embedding_dim': 16},
  'dev_id': {'vocabulary_size': 61, 'embedding_dim': 16},
  'inter_type_cd': {'vocabulary_size': 4, 'embedding_dim': 16},
  'slot_id': {'vocabulary_size': 12, 'embedding_dim': 16},
  'spread_app_id': {'vocabulary_size': 79, 'embedding_dim': 16},
  'tags': {'vocabulary_size': 32, 'embedding_dim': 16},
  'app_first_class': {'vocabulary_size': 3, 'embedding_dim': 16},
  'app_second_class': {'vocabulary_size': 19, 'embedding_dim': 16},
  'age': {'vocabulary_size': 8, 'embedding_dim': 16},
  'city': {'vocabulary_size': 344, 'embedding_dim': 16},
  'city_rank': {'vocabulary_size': 4, 'embedding_dim': 16},
  'device_name': {'vocabulary_size': 94, 'embedding_dim': 16},
  'device_size': {'vocabulary_size': 231, 'embedding_dim': 16},
  'career': {'vocabulary_size': 9, 'embedding_dim': 16},
  'gender': {'vocabulary_size': 3, 'embedding_dim': 16},
  'net_type': {'vocabulary_size': 5, 'embedding_dim': 16},
  'residence': {'vocabulary_size': 36, 'embedding_dim': 16},
  'his_app_size': {'vocabulary_size': 21, 'embedding_dim': 16},
  'his_on_shelf_time': {'vocabulary_size': 4, 'embedding_dim': 16},
  'app_score': {'vocabulary_size': 2, 'embedding_dim': 16},
  'emui_dev': {'vocabulary_size': 18, 'embedding_dim': 16},
  'list_time': {'vocabulary_size': 19, 'embedding_dim': 16},
  'device_price': {'vocabulary_size': 8, 'embedding_dim': 16},
  'up_life_duration': {'vocabulary_size': 21, 'embedding_dim': 16},
  'up_membership_grade': {'vocabulary_size': 4, 'embedding_dim': 16},
  'membership_life_duration': {'vocabulary_size': 21, 'embedding_dim': 16},
  'consume_purchase': {'vocabulary_size': 9, 'embedding_dim': 16},
  'communication_avgonline_30d': {'vocabulary_size': 14, 'embedding_dim': 16},
  'indu_name': {'vocabulary_size': 42, 'embedding_dim': 16}}}

In [4]:
# class CIN(Layer):
#     def __init__(self, layer_size=(128, 128), activation='relu', split_half=True, l2_reg=0, seed=1024, **kwargs):
#         self.layer_size = layer_size
#         self.split_half = split_half
#         self.activation = activation
#         self.l2_reg = l2_reg
#         self.seed = seed
#         super(CIN, self).__init__(**kwargs)

#     def build(self, input_shape):
#         self.field_nums = [int(input_shape[1])]
#         self.filters = []
#         self.bias = []
#         for i, size in enumerate(self.layer_size):

#             self.filters.append(self.add_weight(name='filter' + str(i),
#                                                 shape=[1, self.field_nums[-1]
#                                                        * self.field_nums[0], size],
#                                                 dtype=tf.float32, initializer=glorot_uniform(
#                     seed=self.seed + i)))

#             self.bias.append(self.add_weight(name='bias' + str(i), shape=[size], dtype=tf.float32,
#                                              initializer=tf.keras.initializers.Zeros()))

#             if self.split_half:
#                 if i != len(self.layer_size) - 1 and size % 2 > 0:
#                     raise ValueError(
#                         "layer_size must be even number except for the last layer when split_half=True")
#                 self.field_nums.append(size // 2)
#             else:
#                 self.field_nums.append(size)

#         self.activation_layers = [Activation(self.activation) for _ in self.layer_size]

#         super(CIN, self).build(input_shape)  # Be sure to call this somewhere!

#     def call(self, inputs, **kwargs):
#         dim = int(inputs.get_shape()[-1])
#         hidden_nn_layers = [inputs]
#         final_result = []

#         split_tensor0 = tf.split(hidden_nn_layers[0], dim * [1], 2)
#         for idx, layer_size in enumerate(self.layer_size):
#             split_tensor = tf.split(hidden_nn_layers[-1], dim * [1], 2)

#             dot_result_m = tf.matmul(
#                 split_tensor0, split_tensor, transpose_b=True)

#             dot_result_o = tf.reshape(
#                 dot_result_m, shape=[dim, -1, self.field_nums[0] * self.field_nums[idx]])

#             dot_result = tf.transpose(dot_result_o, perm=[1, 0, 2])

#             curr_out = tf.nn.conv1d(
#                 dot_result, filters=self.filters[idx], stride=1, padding='VALID')

#             curr_out = tf.nn.bias_add(curr_out, self.bias[idx])

#             curr_out = self.activation_layers[idx](curr_out)

#             curr_out = tf.transpose(curr_out, perm=[0, 2, 1])

#             if self.split_half:
#                 if idx != len(self.layer_size) - 1:
#                     next_hidden, direct_connect = tf.split(
#                         curr_out, 2 * [layer_size // 2], 1)
#                 else:
#                     direct_connect = curr_out
#                     next_hidden = 0
#             else:
#                 direct_connect = curr_out
#                 next_hidden = curr_out

#             final_result.append(direct_connect)
#             hidden_nn_layers.append(next_hidden)

#         result = tf.concat(final_result, axis=1)
#         result = tf.reduce_sum(result, -1, keep_dims=False)

#         return result

#     def compute_output_shape(self, input_shape):
#         if self.split_half:
#             featuremap_num = sum(
#                 self.layer_size[:-1]) // 2 + self.layer_size[-1]
#         else:
#             featuremap_num = sum(self.layer_size)
#         return (None, featuremap_num)

#     def get_config(self, ):

#         config = {'layer_size': self.layer_size, 'split_half': self.split_half, 'activation': self.activation,
#                   'seed': self.seed}
#         base_config = super(CIN, self).get_config()
#         return dict(list(base_config.items()) + list(config.items()))
    
class FM(Layer):
    def __init__(self, **kwargs):

        super(FM, self).__init__(**kwargs)

    def build(self, input_shape):

        super(FM, self).build(input_shape) 

    def call(self, inputs, **kwargs):
        
        ad_group = tf.reduce_sum(inputs[0], axis=1)
        user_group = tf.reduce_sum(inputs[1], axis=1)
        cross_term = tf.reduce_sum(ad_group*user_group,axis=1, keep_dims=True)
        return cross_term

    def compute_output_shape(self, input_shape):
        return (None, 1)
class Add(Layer):
    def __init__(self, **kwargs):
        super(Add, self).__init__(**kwargs)
    def call(self, input, **kwargs):
        return input[0]+input[1]

In [5]:
class model:
    def __init__(self,params):
        '''
        features : 特征名
        '''
        self.params = params
        self.input_dict,self.embedding_dict,self.embedding_lookup = self.build_input(self.params['feature_config'])
        self.model = self.build_model(self.input_dict,self.embedding_lookup,self.embedding_dict)
   #     print(self.model.summary())
        
    def build_input(self,features):
        input_dict = {}
        embedding_dict = {}
        embedding_lookup = {}
        for fea in features:
            feature_embedding = Embedding(
                features[fea]['vocabulary_size'],features[fea]['embedding_dim'],
                embeddings_initializer=RandomNormal(mean=0.0, stddev=0.0001, seed=1996),
                name="emb_" + fea)
            embedding_dict[fea] = feature_embedding
            fea_input = Input(shape=(1,),name = fea)
            input_dict[fea] = fea_input
            embedding = feature_embedding(fea_input)
            embedding_lookup[fea] = embedding
        input_dict['his_adv_id'] = Input(shape=(20,),name = 'his_adv_id')
        input_dict['his_adv_prim_id'] = Input(shape=(20,),name = 'his_adv_prim_id')
      #  input_dict['advid_ld'] = Input(shape=(20,),name = 'advid_ld')
        
        return input_dict,embedding_dict,embedding_lookup
    '''
    ,'slot_id'
    '''
    def build_model(self,input_dict,embedding_lookup,embedding_dict):
       # FMinput = concatenate([embedding_lookup[i] for i in embedding_lookup],axis = 1)
       # exFM_out = CIN()(FMinput)
       # exFM_logit = Dense(1, activation=None, )(exFM_out)
        
        ad_group = concatenate([embedding_lookup[i] for i in ['adv_id','task_id','creat_type_cd','adv_prim_id','dev_id',
                            'inter_type_cd','spread_app_id','tags','app_first_class','app_second_class','his_app_size','his_on_shelf_time',
                                                             'app_score','indu_name']],axis = 1)
        
        user_group = concatenate([embedding_lookup[i] for i in ['age','city','city_rank','device_name','device_size',
                            'career','gender','net_type','residence','emui_dev','list_time','device_price','up_life_duration','up_membership_grade',
                                        'membership_life_duration','consume_purchase','communication_avgonline_30d']],axis = 1)
        
        
        FMout = FM(name = 'FM')([ad_group,user_group])
        his_adv_id = embedding_dict['adv_id'](input_dict['his_adv_id'])
        his_adv_id = Bidirectional(CuDNNGRU(128))(his_adv_id)
        his_adv_prim_id = embedding_dict['adv_prim_id'](input_dict['his_adv_prim_id'])
        his_adv_prim_id = Bidirectional(CuDNNGRU(128))(his_adv_prim_id)
     #   user_app_rep = embedding_dict['adv_id'](input_dict['advid_ld'])
     #   user_app_rep = Bidirectional(CuDNNGRU(128))(user_app_rep)
     #   his_adv_id0 = embedding_dict['adv_id'](input_dict['his_adv_id0'])
     #   his_adv_id0 = Bidirectional(CuDNNGRU(128))(his_adv_id0)
        embedding_concat = Flatten()(concatenate([embedding_lookup[i] for i in embedding_lookup]))
        embedding_concat=concatenate([embedding_concat,his_adv_id,his_adv_prim_id,])#user_app_rep
      #  linear = Dense(1)(his_adv_id0)
        dense = Dropout(0.3)(embedding_concat)
        dense = Dense(1024)(dense)
        dense = BatchNormalization()(dense)
        dense = Activation(activation="relu")(dense)
        dense = Dropout(0.3)(dense)
        dense = Dense(512)(dense)
        dense = BatchNormalization()(dense)
        dense = Activation(activation="relu")(dense)
        dense = Dense(1)(dense)
        FMout = Dense(1)(FMout)
        output = Add()([dense,FMout])#exFM_logit
        output = Activation(activation="sigmoid")(output)
        model = Model(inputs=[input_dict[i] for i in input_dict], outputs=output)
        model = multi_gpu_model(model, gpus=4)
        model.compile(optimizer ='adam',
                      loss= 'binary_crossentropy',
                      metrics=['acc'])
        return model
        
    def train(self,train_data,valid_data,train_label,valid_label,batch_size,epochs,callbacks = None):
        self.model.fit(train_data,train_label, batch_size=batch_size, epochs=epochs, verbose=1, 
                  validation_data=(valid_data, valid_label),callbacks=callbacks)

    def predict(self,test_data,batch_size=4096):
        result = self.model.predict(test_data, batch_size=batch_size)
        return result

In [15]:
def make_data(ind):
    data=dict()
    for i in params['feature_config']:
        data[i] = np.array(train_data[i][ind])
    data['his_adv_id'] = train_his_advid[ind,:]
    data['his_adv_prim_id'] = train_his_adv_prim_id[ind,:]
   # data['advid_ld'] = train_advid_ld[ind,:]
    return data
def make_train():
    data=dict()
    for i in params['feature_config']:
        data[i] = np.array(train_data[i][0:35897957])
    data['his_adv_id'] = train_his_advid[0:35897957,:]
    data['his_adv_prim_id'] = train_his_adv_prim_id[0:35897957,:]
  #  data['his_adv_id0'] = train_his_advid0[0:35897957,:]
    return data
def make_valid():
    data=dict()
    for i in params['feature_config']:
        data[i] = np.array(train_data[i][35897957:])
    data['his_adv_id'] = train_his_advid[35897957:,:]
    data['his_adv_prim_id'] = train_his_adv_prim_id[35897957:,:]
 #   data['his_adv_id0'] = train_his_advid0[35897957:,:]
    return data
def make_test():
    data=dict()
    for i in params['feature_config']:
        data[i] = np.array(test_data[i])
    data['his_adv_id'] = test_his_advid
    data['his_adv_prim_id'] = test_his_adv_prim_id
   # data['advid_ld'] = test_advid_ld
 #   data['his_task_id'] = test_his_task_id
    return data

In [16]:
target = train_data['label']
mode = 1
stack_test = np.zeros((len(test_data),1))
stack_train = np.zeros((train_data.shape[0], 1))
if mode == 1:
    skf = StratifiedKFold(n_splits=5, random_state=1996, shuffle=True)
    for index, (train_index, test_index) in enumerate(skf.split(train_data, target)):
        K.clear_session()
        filepath = "ffm_model%d.h5" % index
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=1, min_lr=0.0001, verbose=1)
        earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2,verbose=1, mode='auto')
        callbacks = [checkpoint, reduce_lr, earlystopping]
        nn_model = model(params)
        trian_x = make_data(train_index)
        valid_x = make_data(test_index)
        trian_y = np.array(target[train_index])
        valid_y = np.array(target[test_index])
        nn_model.train(trian_x,valid_x,trian_y,valid_y,4096,10,callbacks=callbacks,)
        nn_model.model.load_weights(filepath)
        stack_test += nn_model.predict(make_test())
        stack_train[test_index] = nn_model.predict(valid_x)
        
        if index==0:
            
            reslgb = pd.read_csv('lgbres.csv')
            reslgb['probability']=stack_test[1000000:]
            reslgb[['id','probability']].to_csv('subnn.csv',index=False)
else:
    trian_x = make_train()
    valid_x = make_valid()
    trian_y = np.array(target[0:35897957])
    valid_y = np.array(target[35897957:])
    model.train(trian_x,valid_x,trian_y,valid_y,4096,1)






Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 33525706 samples, validate on 8381427 samples
Epoch 1/10
33525706/33525706 [==============================] - 678s 20us/step - loss: 0.1348 - acc: 0.9653 - val_loss: 0.1331 - val_acc: 0.9655

Epoch 00001: val_loss improved from inf to 0.13312, saving model to ffm_model0.h5
Epoch 2/10
33525706/33525706 [==============================] - 660s 20us/step - loss: 0.1328 - acc: 0.9655 - val_loss: 0.1325 - val_acc: 0.9655

Epoch 00002: val_loss improved from 0.13312 to 0.13252, saving model to ffm_model0.h5
Epoch 3/10
33525706/33525706 [==============================] - 661s 20us/step - loss: 0.1323 - acc: 0.9655 - val_loss: 0.1324 - val_acc: 0.9655

Epoch 00003: val_loss improved from 0.132

In [19]:
oof_train = pd.DataFrame(stack_train)
oof_test = pd.DataFrame(stack_test[1000000:]/5)
pd.concat([oof_train,oof_test]).to_hdf('nn1oof.h5', key='data')

In [28]:
from sklearn import metrics

print('val_auc:',metrics.roc_auc_score(valid_y,valid_prob))#0.42480
#val_auc: 0.7650400675874388  0.776936
#val_auc: 0.765866672319938  0.779859
#val_loss: 0.1322 val_auc: 0.7660568973066079  0.780272

val_auc: 0.7651491221203367


In [ ]:
'''
val_auc: 0.8127748523069542   
val_auc: 0.788409133403723   0.780272

'''